本教程将介绍如何使用在线端点部署机器学习模型。您将首先在本地机器上部署模型以调试任何错误，然后您将在 Azure 中部署和测试它。

### 前置条件

**要求**
- 对机器学习有基本了解
- 拥有具有活跃订阅的Azure账户 - [免费创建账户](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- 拥有带计算集群的Azure ML工作区 - [配置工作区](https://aka.ms/azureml-workspace-configuration)
- Python环境
- 已安装Azure机器学习Python SDK v2 - [安装说明](https://aka.ms/azureml-sdkv2-install) - 查看入门部分
* 连接到AzureML工作区。在[设置SDK认证](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk)了解更多。
* 连接到 `azureml` 系统注册表

## 1. 配置在线部署

我们创建 `deployment.yml` 文件配置部署相关设置，并且部署模型


```yaml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: blue
app_insights_enabled: true
endpoint_name: <endpoint name>
model: 
  path: ./model/sklearn_mnist_model.pkl
code_configuration:
  code: ./script/
  scoring_script: score.py
instance_type: <instance type name>
environment: 
  conda_file: file:./model/conda.yml
  image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest
```

参数说明：

1. **基本配置参数**：
- `$schema`: 指定配置文件的JSON模式验证地址，确保YAML格式符合Azure ML规范
- `name`: 部署实例的名称标识符，此处设为"blue"
- `endpoint_name`: 在线端点的名称，需要替换为实际的端点名称

2. **监控与日志**:
- `app_insights_enabled`: 启用Application Insights监控服务，用于收集部署性能和日志数据

3. **模型配置**:
- `model.path`: 指定训练好的模型文件路径，此处为sklearn MNIST模型的pickle文件

4. **代码配置**
- `code_configuration.code`: 包含推理代码的目录路径
- `code_configuration.scoring_script`: 推理脚本文件名，负责模型加载和预测逻辑

5. **基础设施配置**
- `instance_type`: 部署使用的虚拟机SKU类型，需要根据性能需求选择合适规格
- `environment.conda_file`: Conda环境依赖文件路径，定义运行环境的Python包依赖
- `environment.image`: 基础Docker镜像，此处使用Microsoft提供的OpenMPI Ubuntu镜像

下面执行部署命令

In [ ]:
!az ml online-deployment create -f deployment.yml --all-traffic

## 2. 测试端点

首先我们获取端点的评分 URI 和 API 密钥:

In [ ]:
import requests
import json

url = "https://vllm-hf-v1.eastus2.inference.ml.azure.com/v1/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer xxxxxxxxxxxxxx"
}
data = {
    "model": "meta-llama/Llama-3.1-8B-Instruct",
    "prompt": "San Francisco is a",
    "max_tokens": 200,
    "temperature": 0.7
}

response = requests.post(url, headers=headers, json=data)

json_formatted_str = json.dumps(response.json(), indent=4)
print(json_formatted_str)

## 3. 获取日志
获取部署的日志并根据需要进行验证

In [ ]:
!az ml online-deployment get-logs -n blue --endpoint vllm-hf-v1 --lines 50

## 4. 删除端点

In [ ]:
!az ml online-endpoint delete -n vllm-hf-v1 -y